# Marangoni cas test pour IJK FT


## Introduction
 
 Validation made by: Antoine du Cluzeau

 Report generated 31/01/2024

In [ ]:
from trustutils import run 
from pathlib import Path
 
run.TRUST_parameters("1.9.4_beta")

In [ ]:
run.reset()
run.initBuildDirectory()  # copy src/ in build/
run.addCase(".", "Original_interfacial_source.data", nbProcs=8) 
run.addCase(".", "Original_interfacial_source_et_marangoni.data", nbProcs=8) 
run.addCase(".", "Tryggvason_interfacial_source.data", nbProcs=8) 
run.addCase(".", "Tryggvason_interfacial_source_et_marangoni.data", nbProcs=8) 
run.printCases()
run.runCases()

## Computer Performance

In [ ]:
run.tablePerf()

## Resultats


In [ ]:
from trustutils import visit
from math import *
import re
import os
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
from matplotlib.colors import ListedColormap, BoundaryNorm
import sys
from collections import Counter
import matplotlib.patches as mpatches
print(os.getcwd())
sys.path.append(os.getcwd())
from Fields import *
import glob
import turtle

In [ ]:
def param_image(name, grid=True, legend=True, legend_out=True, n_dpi=100, fontsizelegend=10):
    scalepng=3.5       
    inSizeLegend=int(scalepng*3.5)
    if (grid):
        plt.grid()
    plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
    plt.tick_params(axis='x', labelsize=12)
    plt.tick_params(axis='y', labelsize=12)
    if (legend_out):
    	plt.legend(fontsize=fontsizelegend, loc='center left', bbox_to_anchor=(1.05, 0.5))
    else:
    	plt.legend(fontsize=fontsizelegend, loc=0,prop={'size':inSizeLegend}, frameon=True, ncol=1)
    	plt.legend(fontsize=fontsizelegend).set_visible(legend) ### deactive la legende
        
def trouver_dernier_fichier(repertoire, debut):
    fichiers = glob.glob(os.path.join(repertoire, f"{debut}*"))

    # Filtrer uniquement les fichiers (exclure les dossiers)
    fichiers = [f for f in fichiers if os.path.isfile(f)]

    if not fichiers:
        return None  # Retourner None si aucun fichier correspondant n'a été trouvé

    # Trouver le fichier le plus récent
    dernier_fichier = max(fichiers, key=os.path.getctime)
    return dernier_fichier

On réalise un cas test du terme source de Marangoni selon Marudoglu et Tryggvason (2007), voir aussi thèse de Kalyani (p.88)
On impose une concentration de surfactant
\begin{equation}
\sigma(z)= \sigma_0(1-\beta\frac{z-z_b}{l_z})
\end{equation}
$z$ est la hauteur, $z_b$ la hauteur du centre de gravité de la bulle. $l_z$ est la hauteur du domaine numérique.
On peut alors montrer que pour $Ca=\mu_lu_r/\sigma_0$ très petit, et pour $Re=\rho_lr_bv_r/\mu_l$ très petit, il existe une solution analytique. La vitesse terminale de la bulle générée par effet Marangoni est :
\begin{equation}
V_{YBG}=\frac{2\sigma_0\beta r_b}{l_z(6\mu_l+9\mu_v)}
\end{equation}

In [ ]:
V = 4.188666666666667e-12 
a = (V/((4./3.)*3.1415))**(1./3.)
L = 0.001
beta = 5.
sigma0 = 0.0002
mu_v = 2.944271909999159e-03
mu_d = 2.944271909999159e-03
rhov = 1000.
rhod = 1000.
repertoire = "build/"
debut = "diphasique_moyenne"


UI=Field.LoadFromFile(trouver_dernier_fichier(repertoire, debut), 0, 2)
I=Field.LoadFromFile(trouver_dernier_fichier(repertoire, debut), 0, 1)
U = UI/I
Ul = U.integrale()

Vybg = 2. * sigma0 * beta * a /  (L * ( 6 * mu_v + 9 * mu_d))

position=Field.LoadFromFile(repertoire+"Original_interfacial_source_bulles_centre_x.out", 0, 1)
velocity=position.gradient()-Ul
Re = rhod * a * velocity.npa[velocity.npa.size-2] / mu_d
Ca = mu_v * velocity.npa[velocity.npa.size-2] / sigma0
print("Re terminal = ", Re)
print("Ca terminal = ", Ca)
print(" ")
velocity = velocity / Vybg

plt.figure()
plt.plot(velocity.ax, velocity.npa, color='red', label="original", linestyle='-')

position=Field.LoadFromFile(repertoire+"Original_interfacial_source_et_marangoni_bulles_centre_x.out", 0, 1)
velocity=position.gradient()-Ul
velocity = velocity / Vybg
plt.plot(velocity.ax, velocity.npa, color='blue', label="original + marangoni", linestyle='-')

position=Field.LoadFromFile(repertoire+"Tryggvason_interfacial_source_bulles_centre_x.out", 0, 1)
velocity=position.gradient()-Ul
velocity = velocity / Vybg
plt.plot(velocity.ax, velocity.npa, color='green', label="Tryggvason", linestyle='-')

position=Field.LoadFromFile(repertoire+"Tryggvason_interfacial_source_et_marangoni_bulles_centre_x.out", 0, 1)
velocity=position.gradient()-Ul
velocity = velocity / Vybg
plt.plot(velocity.ax, velocity.npa, color='brown', label="Tryggvason + marangoni", linestyle='-')

velocity = velocity * 0. + 1.
plt.plot(velocity.ax, velocity.npa, color='black', label="Solution Young et al.", linestyle='-')

plt.legend(fontsize=10, loc=0,prop={'size':10}, frameon=True, ncol=1)
plt.legend(fontsize=10).set_visible(True) ### deactive la legende
plt.ylabel(r"Terminal Velocity ($V_t/V_{YBG}$)", fontsize=15)
plt.xlabel("Time (t)", fontsize=15)
plt.ylim(0.,1.2)
plt.show()


On s'assure par vérification visuel de la régularité du champ en fin de calcul que tout s'est bien déroulé. 

In [ ]:
visit.showField("Original_interfacial_source.lata", "Pseudocolor", "GRADX_SIGMA_SOM_INTERFACES", plotmesh=False)

In [ ]:
visit.showField("Original_interfacial_source_et_marangoni.lata", "Pseudocolor", "GRADX_SIGMA_SOM_INTERFACES", plotmesh=False)

In [ ]:
visit.showField("Tryggvason_interfacial_source.lata", "Pseudocolor", "GRADX_SIGMA_SOM_INTERFACES", plotmesh=False)

In [ ]:
visit.showField("Tryggvason_interfacial_source_et_marangoni.lata", "Pseudocolor", "GRADX_SIGMA_SOM_INTERFACES", plotmesh=False)